# Parametrized circuits

This tutorial shows you can submit paranetrized quantum circuits to the `POVMSampler`.

In [1]:
%load_ext autoreload
%autoreload 2

## Parametrized Circuit

Let us look at a 2-qubit quantum circuit with 5 parameters.

In [2]:
from qiskit.circuit.library import RealAmplitudes

# Prepare inputs.
n_qubit=2
qc = RealAmplitudes(num_qubits=n_qubit, reps=2)
qc.draw()

┌────────────────────────────────────────────────┐
q_0: ┤0                                               ├
     │  RealAmplitudes(θ[0],θ[1],θ[2],θ[3],θ[4],θ[5]) │
q_1: ┤1                                               ├
     └────────────────────────────────────────────────┘

In [3]:
import numpy as np

# Assume you want to run the circuit with two different sets of parameter values
theta = np.array([
    [0, 1, 1, 2, 3, 5],  # first set of parameter values
    [0, 1, 1, 2, 2, 5],  # second set of parameter values
    ])
# Shape of the resulting `BindingsArray` is (2,) with `num_param` equal to 6

## Measurement

We now look at the implementation of Classical Shaodws measurement

In [4]:
from povm_toolbox.library import ClassicalShadows

# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the default shot budget.
shots = 4096

In [5]:
pub = (qc, theta, shots, cs_implementation)

In [6]:
from povm_toolbox.sampler import POVMSampler
from qiskit.primitives import StatevectorSampler as Sampler

# Internal `BaseSampler`
sampler = Sampler()

# Actual `POVMSampler` instance
povm_sampler = POVMSampler(sampler=sampler)

# Submit the job by specifying the list of PUBs to run, here we have a single PUB.
job = povm_sampler.run([pub])

### Results
We submitted a single PUB, hence the `PrimitiveResult` will contain only one `POVMPubResult`.

In [7]:
pub_result = job.result()[0]
print(pub_result)

# Note that the pub result will contain a `BitArray` that has the same shape
# as the submitted `BindingsArray`, which is (2,) in this example.
print(pub_result.get_counts().shape)

POVMPubResult(data=DataBin<2>(povm_measurement_creg=BitArray(<shape=(2,), num_shots=4096, num_bits=2>)), metadata=RandomizedPMsMetadata(povm=ClassicalShadows(n_qubits=2), pvm_keys=np.ndarray<2,4096,2>))
(2,)


We now define our POVM post-processor, which will use the result object to estimate expectation values of some observables.

In [8]:
from povm_toolbox.post_processor.povm_post_processor import POVMPostprocessor

post_processor = POVMPostprocessor(pub_result)

Since the POVM implementation that we used is informationally complete, we can define our observables of interest after the sampling process.

In [9]:
from qiskit.quantum_info import SparsePauliOp

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("II", 1), ("XX", 1), ("YY", -1), ("ZZ", 1)])

The post-processor will return two expectation values corresponding to the two different sets of parameter values that were submitted.

In [10]:
exp_value = post_processor.get_expectation_value(H1)
print(exp_value)

[1.72070312 2.00268555]


If we are interested to evaluate an observable only for one set of parameter values, this set can be specified through the `loc` argument.

In [11]:
exp_value = post_processor.get_expectation_value(H2, loc=1)
print(exp_value)

0.15844726562500044


## Further example
Let us look at a `BindingsArray` instance with a more complex shape.

In [12]:
bindings_array_shape = (3,4)
num_param = 6

theta = np.arange(72).reshape((*bindings_array_shape,num_param))

In [13]:
job = povm_sampler.run([(qc, theta, shots, cs_implementation)])
pub_result = job.result()[0]
print(pub_result.get_counts().shape)

(3, 4)


In [14]:
post_processor = POVMPostprocessor(pub_result)
exp_values = post_processor.get_expectation_value(H1)
print(exp_values)

[[-0.93359375  0.46899414  1.80273438  1.68701172]
 [ 0.70483398  0.81323242  2.37548828  4.0168457 ]
 [ 4.99975586  4.38745117  3.11083984  2.00927734]]
